# Pandas基礎② - データクリーニング（Excelユーザー向けシンプル版）

Day 4 - Pandas Data Cleaning Basic Simple

このノートブックでは、Excelユーザーがデータクリーニングの基本を学ぶためのシンプル版を提供します。
実務でよく使うデータクリーニング手法を、分かりやすく解説しています。

In [1]:
import pandas as pd
import numpy as np

print("=== Pandas データクリーニング基礎（シンプル版） ===")

=== Pandas データクリーニング基礎（シンプル版） ===


## 1. サンプルデータの作成（よくある問題を含むデータ）

実務でよく遭遇する問題を含んだ顧客データを作成します。

In [2]:
# 顧客データのサンプル（実務でよくある問題を含む）
data = {
    '顧客ID': ['C001', 'C002', 'C003', 'C004', 'C005', 'C006', 'C007'],
    '名前': ['山田太郎', '佐藤花子', np.nan, '鈴木一郎', '田中美咲', '高橋健太', '伊藤さくら'],
    '年齢': [25, 32, 45, np.nan, 28, 35, 29],
    'メール': ['yamada@email.com', 'sato@email.com', 'tanaka@email.com', 
               'suzuki@email.com', 'tanaka@email.com', None, 'ito@email.com'],
    '購入金額': ['1,500', '2,300', '980', '3,200', '1,800', '2,500', '1,200'],
    '会員ランク': ['Bronze', 'Silver', 'bronze', 'Gold', 'Silver', 'GOLD', 'Bronze']
}

df = pd.DataFrame(data)
print("元のデータ:")
df

元のデータ:


,顧客ID,名前,年齢,メール,購入金額,会員ランク
0,C001,山田太郎,25.0,yamada@email.com,"1,500",Bronze
1,C002,佐藤花子,32.0,sato@email.com,"2,300",Silver
2,C003,NaN,45.0,tanaka@email.com,980,bronze
3,C004,鈴木一郎,NaN,suzuki@email.com,"3,200",Gold
4,C005,田中美咲,28.0,tanaka@email.com,"1,800",Silver
5,C006,高橋健太,35.0,None,"2,500",GOLD
6,C007,伊藤さくら,29.0,ito@email.com,"1,200",Bronze


## 2. 欠損値（空白セル）の確認

Excelでいう「空白セル」をPandasでは「欠損値（NaN）」と呼びます。

In [4]:
# df.isnull()は各セルが欠損値かどうかのBoolean値（True/False）を返します
print("各列の欠損値の数:")
print(df.isnull().sum())
print()

# axis=1は「行方向」を指定します（axis=0は列方向）
# any()は「 1つでも True があれば True 」を返す関数です
print("欠損値を含む行だけを表示:")
df[df.isnull().any(axis=1)]

各列の欠損値の数:
顧客ID     0
名前       1
年齢       1
メール      1
購入金額     0
会員ランク    0
dtype: int64

欠損値を含む行だけを表示:


,顧客ID,名前,年齢,メール,購入金額,会員ランク
2,C003,NaN,45.0,tanaka@email.com,980,bronze
3,C004,鈴木一郎,NaN,suzuki@email.com,"3,200",Gold
5,C006,高橋健太,35.0,None,"2,500",GOLD


## 3. 欠損値の処理方法① - 削除

重要なデータが欠けている場合は、その行を削除します。

In [5]:
# 名前が欠損している行を削除
df_cleaned1 = df.dropna(subset=['名前'])
print("名前が空白の行を削除後:")
df_cleaned1

名前が空白の行を削除後:


,顧客ID,名前,年齢,メール,購入金額,会員ランク
0,C001,山田太郎,25.0,yamada@email.com,"1,500",Bronze
1,C002,佐藤花子,32.0,sato@email.com,"2,300",Silver
3,C004,鈴木一郎,NaN,suzuki@email.com,"3,200",Gold
4,C005,田中美咲,28.0,tanaka@email.com,"1,800",Silver
5,C006,高橋健太,35.0,None,"2,500",GOLD
6,C007,伊藤さくら,29.0,ito@email.com,"1,200",Bronze


## 4. 欠損値の処理方法② - 穴埋め

数値データの場合は、平均値や中央値で穴埋めすることができます。

In [6]:
# 年齢の欠損値を平均値で埋める
# fillna()は「欠損値（NaN）を指定した値で埋める」メソッドです。
df_copy = df.copy()
average_age = df_copy['年齢'].mean()
df_copy['年齢'] = df_copy['年齢'].fillna(average_age)

# df_copy['年齢']：年齢列を取得
# .fillna(average_age)：その列の欠損値をaverage_age（平均値）で置き換え
# df_copy['年齢'] = ：結果を元の年齢列に代入

print(f"年齢の平均値: {average_age:.1f}")
print("\n年齢の空白を平均値で埋めた後:")
df_copy[['顧客ID', '名前', '年齢']]

年齢の平均値: 32.3

年齢の空白を平均値で埋めた後:


,顧客ID,名前,年齢
0,C001,山田太郎,25.000000
1,C002,佐藤花子,32.000000
2,C003,NaN,45.000000
3,C004,鈴木一郎,32.333333
4,C005,田中美咲,28.000000
5,C006,高橋健太,35.000000
6,C007,伊藤さくら,29.000000


## 5. 重複データの確認と削除

同じメールアドレスが複数回登録されているケースを確認します。

In [7]:
# duplicated()は「重複している行を検出する」メソッド
# keep=False
#keepパラメータは「重複のうちどれを残すか」を指定します：

# keep='first'（デフォルト）：最初の行を残し、2番目以降を True とする
# keep='last'：最後の行を残し、それ以前を True とする
# keep=False：重複するすべての行を True とする

print("メールアドレスの重複チェック:")
print(df[df.duplicated(subset=['メール'], keep=False)])
print()

# 重複を削除（最初のものを残す）
df_no_dup = df.drop_duplicates(subset=['メール'], keep='first')
print("重複メールアドレスを削除後:")
df_no_dup[['顧客ID', '名前', 'メール']]

メールアドレスの重複チェック:
   顧客ID    名前    年齢               メール   購入金額   会員ランク
2  C003   NaN  45.0  tanaka@email.com    980  bronze
4  C005  田中美咲  28.0  tanaka@email.com  1,800  Silver

重複メールアドレスを削除後:


,顧客ID,名前,メール
0,C001,山田太郎,yamada@email.com
1,C002,佐藤花子,sato@email.com
2,C003,NaN,tanaka@email.com
3,C004,鈴木一郎,suzuki@email.com
5,C006,高橋健太,None
6,C007,伊藤さくら,ito@email.com


## 6. データ型の変換（文字列→数値）

カンマ付きの数値を、計算可能な数値型に変換します。

In [8]:
# カンマ付き数値を数値型に変換
df_copy2 = df.copy()
df_copy2['購入金額'] = df_copy2['購入金額'].str.replace(',', '').astype(float)

# .str アクセサ
# 役割： 文字列操作のためのアクセサ
# pythondf['列名'].str
# pandasのSeriesが文字列型の場合に使用
# 文字列に対する様々な操作を可能にする
# 重要： .str自体はメソッドではなく「文字列操作へのアクセス手段」

print("購入金額を数値型に変換後:")
print(df_copy2[['顧客ID', '購入金額']])
print(f"\n購入金額の合計: {df_copy2['購入金額'].sum():,.0f}円")

# {値:,.0f}
# : フォーマット指定の開始
# , 3桁区切りのカンマを表示
# . 0小数点以下0桁（整数表示）
# f 浮動小数点数として表示

購入金額を数値型に変換後:
   顧客ID    購入金額
0  C001  1500.0
1  C002  2300.0
2  C003   980.0
3  C004  3200.0
4  C005  1800.0
5  C006  2500.0
6  C007  1200.0

購入金額の合計: 13,480円


## 7. 文字列の統一（大文字小文字の統一）

同じ会員ランクなのに、大文字小文字が混在している問題を解決します。

In [14]:
print("変換前の会員ランク:")
print(df['会員ランク'].value_counts())
print()

# 会員ランクを統一（最初を大文字、残りを小文字に）
df_copy3 = df.copy()
df_copy3['会員ランク'] = df_copy3['会員ランク'].str.capitalize()

print("変換後の会員ランク:")
print(df_copy3['会員ランク'].value_counts())

print("\n他の似たメソッド：")
text = "HELLO world"
print(f"capitalize(): {text.capitalize()}")  # Hello world
print(f"title():      {text.title()}")      # Hello World
print(f"lower():      {text.lower()}")      # hello world
print(f"upper():      {text.upper()}")      # HELLO WORLD

変換前の会員ランク:
会員ランク
Bronze    2
Silver    2
bronze    1
Gold      1
GOLD      1
Name: count, dtype: int64

変換後の会員ランク:
会員ランク
Bronze    3
Silver    2
Gold      2
Name: count, dtype: int64

他の似たメソッド：
capitalize(): Hello world
title():      Hello World
lower():      hello world
upper():      HELLO WORLD


## 8. 実践的な例：すべてのクリーニングを組み合わせる

ここまで学んだ技術を組み合わせて、データを一括でクリーニングする関数を作成します。

In [15]:
def clean_customer_data(df):
    """顧客データをクリーニングする関数"""
    # コピーを作成
    cleaned_df = df.copy()
    
    # 1. 名前が欠損している行を削除
    cleaned_df = cleaned_df.dropna(subset=['名前'])
    
    # 2. 年齢の欠損値を平均値で埋める
    avg_age = cleaned_df['年齢'].mean()
    cleaned_df['年齢'] = cleaned_df['年齢'].fillna(avg_age)
    
    # 3. メールの欠損値を'未登録'で埋める
    cleaned_df['メール'] = cleaned_df['メール'].fillna('未登録')
    
    # 4. 購入金額を数値型に変換
    cleaned_df['購入金額'] = cleaned_df['購入金額'].str.replace(',', '').astype(float)
    
    # 5. 会員ランクを統一
    cleaned_df['会員ランク'] = cleaned_df['会員ランク'].str.capitalize()
    
    return cleaned_df

# クリーニング実行
cleaned_data = clean_customer_data(df)
print("クリーニング後の最終データ:")
cleaned_data

クリーニング後の最終データ:


,顧客ID,名前,年齢,メール,購入金額,会員ランク
0,C001,山田太郎,25.0,yamada@email.com,1500.0,Bronze
1,C002,佐藤花子,32.0,sato@email.com,2300.0,Silver
3,C004,鈴木一郎,29.8,suzuki@email.com,3200.0,Gold
4,C005,田中美咲,28.0,tanaka@email.com,1800.0,Silver
5,C006,高橋健太,35.0,未登録,2500.0,Gold
6,C007,伊藤さくら,29.0,ito@email.com,1200.0,Bronze


In [16]:
# クリーニング結果のサマリー
print("クリーニング結果のサマリー:")
print(f"- 削除された行数: {len(df) - len(cleaned_data)}")
print(f"- 欠損値の残り: {cleaned_data.isnull().sum().sum()}")
print(f"- 購入金額の合計: {cleaned_data['購入金額'].sum():,.0f}円")
print(f"- 購入金額の平均: {cleaned_data['購入金額'].mean():,.0f}円")

クリーニング結果のサマリー:
- 削除された行数: 1
- 欠損値の残り: 0
- 購入金額の合計: 12,500円
- 購入金額の平均: 2,083円


## 9. 便利なTips

Excelユーザーのための、よく使うデータクリーニング操作の対応表です。

In [17]:
print("=== 便利なTips ===")
print("-" * 50)

print("1. Excelでよくやる操作とPandasの対応:")
print("   - 空白セルの削除 → dropna()")
print("   - 検索と置換 → replace()")
print("   - 重複の削除 → drop_duplicates()")
print("   - テキストの統一 → str.capitalize(), str.upper(), str.lower()")
print()

print("2. よくあるデータクリーニングのパターン:")
print("   - 数値に混じった文字を除去: str.replace(',', '')")
print("   - 前後の空白を削除: str.strip()")
print("   - 特定の文字で分割: str.split()")
print()

print("3. 欠損値の処理方法の選び方:")
print("   - 重要なデータ → 削除（dropna）")
print("   - 数値データ → 平均値や中央値で穴埋め（fillna）")
print("   - カテゴリデータ → '不明'や'未登録'で穴埋め")

=== 便利なTips ===
--------------------------------------------------
1. Excelでよくやる操作とPandasの対応:
   - 空白セルの削除 → dropna()
   - 検索と置換 → replace()
   - 重複の削除 → drop_duplicates()
   - テキストの統一 → str.capitalize(), str.upper(), str.lower()

2. よくあるデータクリーニングのパターン:
   - 数値に混じった文字を除去: str.replace(',', '')
   - 前後の空白を削除: str.strip()
   - 特定の文字で分割: str.split()

3. 欠損値の処理方法の選び方:
   - 重要なデータ → 削除（dropna）
   - 数値データ → 平均値や中央値で穴埋め（fillna）
   - カテゴリデータ → '不明'や'未登録'で穴埋め


## 練習問題

以下の売上データをクリーニングしてみましょう。

In [18]:
# 練習用データ
practice_data = {
    '商品名': ['りんご', 'バナナ', None, 'みかん', 'りんご'],
    '売上個数': ['100個', '150個', '80個', '200個', '100個'],
    '単価': [150, 80, 120, np.nan, 150],
    '担当者': ['田中', 'TANAKA', '佐藤', '田中', '山田']
}
practice_df = pd.DataFrame(practice_data)
print("練習問題のデータ:")
print(practice_df)
print("\nヒント: 商品名の欠損値処理、売上個数の数値変換、単価の欠損値処理、担当者名の統一を行ってください。")

練習問題のデータ:
    商品名  売上個数     単価     担当者
0   りんご  100個  150.0      田中
1   バナナ  150個   80.0  TANAKA
2  None   80個  120.0      佐藤
3   みかん  200個    NaN      田中
4   りんご  100個  150.0      山田

ヒント: 商品名の欠損値処理、売上個数の数値変換、単価の欠損値処理、担当者名の統一を行ってください。


In [33]:
# ここに解答を書いてみましょう
# practice_df_cleaned = practice_df.copy()
practice_df_cleaned = practice_df.copy()

# 1. 商品名の欠損値処理
practice_df_cleaned = practice_df_cleaned.dropna(subset=['商品名'])

# 2. 売上個数の数値変換
practice_df_cleaned['売上個数'] = practice_df_cleaned['売上個数'].str.replace('個', '').astype(int)

# 3. 単価の欠損値処理（変数名は正確につけること）
average_price = practice_df_cleaned['単価'].mean()
practice_df_cleaned['単価'] = practice_df_cleaned['単価'] .fillna(average_price)

# 4. 担当者名の統一
practice_df_cleaned['担当者'] = practice_df_cleaned['担当者'].str.replace('TANAKA', '田中')

print(practice_df_cleaned)



   商品名  売上個数          単価 担当者
0  りんご   100  150.000000  田中
1  バナナ   150   80.000000  田中
3  みかん   200  126.666667  田中
4  りんご   100  150.000000  山田
